In [1]:
%load_ext cython

In [2]:
import numpy as np

In [3]:
def _embed(x_1d, order, delay):
        """
        Time-delay embedding.
        Parameters
        ----------
        x_1d : 1d-array, shape (n_times)
            Time series
        order : int
            Embedding dimension (order)
        delay : int
            Delay.
        Returns
        -------
        embedded : ndarray, shape (n_times - (order - 1) * delay, order)
            Embedded time-series.
        """
        N = x_1d.size
        Y = np.empty((order, N - (order - 1) * delay))
        for i in range(order):
            Y[i] = x_1d[i * delay:i * delay + Y.shape[1]]
        return Y.T
    
def permen(x, order, delay, normalize=False):
    pe = np.zeros((x.shape[0], x.shape[2]))
    hashmult = np.power(order, np.arange(order))
    
    for i in range(x.shape[0]):
        for k in range(x.shape[2]):
            sorted_idx = _embed(x[i, :, k], order, delay).argsort(kind='quicksort')
            
            # associate unique integer to each permutation
            hashval = np.sum(np.multiply(sorted_idx, hashmult), axis=1)
            
            # return counts
            _, c = np.unique(hashval, return_counts=True)
            
            p = c / c.sum()
            pe[i, k] = -np.multiply(p, np.log2(p)).sum()
    if normalize:
        pe = pe / np.log2(np.factorial(order))
    
    return pe

In [4]:
from perment import embed as fembed

In [5]:
np.random.seed(51)
x = np.random.rand(500, 150, 3)
xf = np.asfortranarray(x.transpose([1, 2, 0]))

In [6]:
%timeit permen(x, 3, 1)

KeyboardInterrupt: 

In [10]:
np.random.seed(5)
y = np.around(np.random.rand(250), 2)

%timeit _embed(y, 3, 1)
%timeit fembed(y, 3, 1)

2.47 µs ± 15.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
438 ns ± 2.21 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
